### Tabelas a serem utilizadas
- results
- drivers
- constructors
- circuits
- races

In [0]:
from pyspark.sql.functions import col, current_timestamp, when

In [0]:
%sql
show tables IN f1_bronze

In [0]:
for column in spark.table("f1_bronze.results").columns:
    print(f"{column}")

In [0]:
COLUMNS_TO_SELECT = [
    "resultId",
    "raceId",
    "driverId",
    "constructorId",
    "number",
    "grid",
    "position",
    "positionText",
    "positionOrder",
    "points",
    "laps",
    "time",
    "milliseconds",
    "fastestLap",
    "rank",
    "fastestLapTime",
    "fastestLapSpeed",
    "bronze_ingestion",
]
df_results = (
    spark.table("f1_bronze.results")
    .select(*COLUMNS_TO_SELECT)
    .withColumn("silver_ingestion", current_timestamp())
    .withColumnRenamed("resultId", "result_id")
    .withColumnRenamed("raceId", "race_id")
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumnRenamed("positionText", "position_text")
    .withColumnRenamed("positionOrder", "position_order")
    .withColumn(
        "rank", when(col("rank") != "\\N", col("rank").cast("int")).otherwise(None)
    )
    .filter(col("rank") <= 10)
)
df_results.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("race_id") \
    .saveAsTable("f1_silver.results")